In [2]:
import requests

def get_card_cv(name_en: str):

    url_en = "https://shadowverse-portal.com/api/v1/cards?format=json&lang=en"
    data_en = requests.get(url_en).json()
    target = None
    
    for card in data_en["data"]["cards"]:
        card_name_en = card.get("card_name")
        if card_name_en and card_name_en.lower() == name_en.lower():
            target = card
            break
    
    if not target:
        return None
    
    card_id = target["card_id"]

    url_ja = "https://shadowverse-portal.com/api/v1/cards?format=json&lang=ja"
    data_ja = requests.get(url_ja).json()
    for card in data_ja["data"]["cards"]:
        if card["card_id"] == card_id:
            return {
                "name": target["card_name"],
                "cv": card.get("cv")
            }

print(get_card_cv("Eleanor, Glorious Flower"))


{'name': 'Eleanor, Glorious Flower', 'cv': '内田真礼'}


In [1]:
import requests

url_en = "https://shadowverse-portal.com/api/v1/cards?format=json&lang=en"
data = requests.get(url_en).json()

cards = data["data"]["cards"]
print("Number of cards:", len(cards))

# Print keys of the first card
print(cards[0].keys())

# Print one sample nicely
import json
print(json.dumps(cards[0], indent=2, ensure_ascii=False))


Number of cards: 5933
dict_keys(['card_id', 'foil_card_id', 'card_set_id', 'card_name', 'card_name_ruby', 'is_foil', 'char_type', 'clan', 'tribe_name', 'skill', 'skill_condition', 'skill_target', 'skill_option', 'skill_preprocess', 'skill_disc', 'org_skill_disc', 'evo_skill_disc', 'org_evo_skill_disc', 'cost', 'atk', 'life', 'evo_atk', 'evo_life', 'rarity', 'get_red_ether', 'use_red_ether', 'description', 'evo_description', 'cv', 'copyright', 'base_card_id', 'normal_card_id', 'format_type', 'restricted_count', 'restricted_count_co_main', 'restricted_count_co_sub', 'resource_card_id', 'resurgent_card', 'is_ts_rotation', 'is_eight_one_zero_start_id'])
{
  "card_id": 930844060,
  "foil_card_id": 930844061,
  "card_set_id": 90000,
  "card_name": "Maisha's Ultimate",
  "card_name_ruby": "",
  "is_foil": 0,
  "char_type": 4,
  "clan": 8,
  "tribe_name": "-",
  "skill": "powerup,quick,bp_modifier",
  "skill_condition": "character=me,{me.inplay.class.bp}>=10,character=me",
  "skill_target": "c

In [2]:
import requests
import json

EN_URL = "https://shadowverse-portal.com/api/v1/cards?format=json&lang=en"
JA_URL = "https://shadowverse-portal.com/api/v1/cards?format=json&lang=ja"

# fetch
cards_en = requests.get(EN_URL).json()["data"]["cards"]
cards_ja = requests.get(JA_URL).json()["data"]["cards"]

# make JP map by id
jp_by_id = {c["card_id"]: c for c in cards_ja}

# pick only fields you care about
slim = []
for c in cards_en:
    cid = c["card_id"]
    jp = jp_by_id.get(cid, {})
    slim.append({
        "card_id": cid,
        "name_en": c.get("card_name"),
        "set_id": c.get("card_set_id"),
        "rarity": c.get("rarity"),
        "cost": c.get("cost"),
        "cv_jp": jp.get("cv"),  # only JP has CV
    })

# preview the first few
for row in slim[:10]:
    print(row)

# save to file
with open("cards_slim.json", "w", encoding="utf-8") as f:
    json.dump(slim, f, ensure_ascii=False, indent=2)
print("Saved cards_slim.json with", len(slim), "entries")


{'card_id': 930844060, 'name_en': "Maisha's Ultimate", 'set_id': 90000, 'rarity': 4, 'cost': -99, 'cv_jp': '-'}
{'card_id': 930144010, 'name_en': "Sekka's Technique", 'set_id': 90000, 'rarity': 1, 'cost': -1, 'cv_jp': '-'}
{'card_id': 930244010, 'name_en': "Taketsumi's Technique", 'set_id': 90000, 'rarity': 3, 'cost': -1, 'cv_jp': '-'}
{'card_id': 930244040, 'name_en': "Bunny's Technique", 'set_id': 90000, 'rarity': 1, 'cost': -1, 'cv_jp': '-'}
{'card_id': 930244070, 'name_en': "Nahtnaught's Technique", 'set_id': 90000, 'rarity': 1, 'cost': -1, 'cv_jp': '-'}
{'card_id': 930344040, 'name_en': "Amaryllis's Technique", 'set_id': 90000, 'rarity': 1, 'cost': -1, 'cv_jp': '-'}
{'card_id': 930344070, 'name_en': "Eleanor's Technique", 'set_id': 90000, 'rarity': 2, 'cost': -1, 'cv_jp': '-'}
{'card_id': 930444010, 'name_en': "Drache's Technique", 'set_id': 90000, 'rarity': 1, 'cost': -1, 'cv_jp': '-'}
{'card_id': 930444070, 'name_en': "Mizuchi's Technique", 'set_id': 90000, 'rarity': 1, 'cost': 

In [13]:
import json
import collections
import requests

EN_URL = "https://shadowverse-portal.com/api/v1/cards?format=json&lang=en"
JA_URL = "https://shadowverse-portal.com/api/v1/cards?format=json&lang=ja"

def fetch_cards(url):
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.json()["data"]["cards"]

cards_en = fetch_cards(EN_URL)
cards_ja = fetch_cards(JA_URL) 

print("EN cards:", len(cards_en), "JP cards:", len(cards_ja))


EN cards: 5933 JP cards: 5933


In [30]:
import json
from pathlib import Path

DATA_DIR = Path("D:/Seiyuu_Search") / "data"
file_path = DATA_DIR / "cards_slim.json"

# Load
with open(file_path, encoding="utf-8") as f:
    cards = json.load(f)

# Clean
fixed_cards = []
for c in cards:
    cv = c.get("cv_jp")
    # Check if cv is not a string or if it is not '-'
    if not isinstance(cv, str) or cv != "-":
        fixed_cards.append(c)

print("Remove Story specialist cards:", len(fixed_cards))

# Save back
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(fixed_cards, f, ensure_ascii=False, indent=2)

print("File updated")

Remove Story specialist cards: 3725
File updated


In [ ]:
import json
from pathlib import Path
from collections import Counter

DATA_DIR = Path("D:/Seiyuu_Search") / "data"

with open(DATA_DIR / "cards_slim.json", encoding="utf-8") as f:
    cards = json.load(f)

# count how many times for each cv_jp
cv_counter = Counter(
    c["cv_jp"] for c in cards
    if c.get("cv_jp") and c["cv_jp"] != "-"
)

print("Unique CV count:", len(cv_counter))


# save to file
with open(DATA_DIR / "unique_cv.txt", "w", encoding="utf-8") as f:
    for cv, count in sorted(cv_counter.items()):
        f.write(f"{cv}\n")

print(f"All unique CVs with counts saved → {DATA_DIR / 'unique_cv.txt'}")


Unique CV count: 555
All unique CVs with counts saved → D:\Seiyuu_Search\data\unique_cv.txt
